In [26]:
# Libraries for Data Visualisation
import os
import cv2
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
from matplotlib.image import imread
from sklearn.metrics import fbeta_score
from tqdm import tqdm
import tensorflow as tf
from keras import optimizers
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input , Dense , Dropout , Flatten,\
Conv2D,MaxPooling2D , BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History
import tensorflow_addons as tfa

In [27]:
path = "../input/planets-dataset/"
os.listdir(path)

['planet', 'test-jpg-additional']

In [28]:
train_label = pd.read_csv("/kaggle/input/planets-dataset/planet/planet/train_classes.csv")
train_label.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [29]:
labels = set()
def splitting_tags(tags):
    '''
    Takes in tags column, splits the tags and store as a set
    '''
    [labels.add(tag) for tag in tags.split()]
    
# Create a copy of train_label
train = train_label.copy()
train['tags'].apply(splitting_tags)
labels = list(labels)
print(labels)

['artisinal_mine', 'road', 'habitation', 'cultivation', 'conventional_mine', 'haze', 'cloudy', 'blooming', 'selective_logging', 'clear', 'slash_burn', 'blow_down', 'primary', 'water', 'agriculture', 'bare_ground', 'partly_cloudy']


In [30]:
##One hot encoding is performed on the labels in train classes 

for tag in labels:
    train[tag] = train['tags'].apply(lambda x: 1 if tag in x.split() else 0)
    
## adding .jpg extension to the column image_name so as to have same name format as the image files
train['image_name'] = train['image_name'].apply(lambda x: '{}.jpg'.format(x))
train.head()

,image_name,tags,artisinal_mine,road,habitation,cultivation,conventional_mine,haze,cloudy,blooming,selective_logging,clear,slash_burn,blow_down,primary,water,agriculture,bare_ground,partly_cloudy
0,train_0.jpg,haze primary,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,train_1.jpg,agriculture clear primary water,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0
2,train_2.jpg,clear primary,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
3,train_3.jpg,clear primary,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
4,train_4.jpg,agriculture clear habitation primary road,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0


In [31]:
#importing libraries for training
import tensorflow as trf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [32]:
# Defining the columns,i.e the labels that were newly added to the train_classes via hot encoding.
columns = list(train.columns[2:])

In [33]:
def fbeta(y_true, y_pred, beta = 2, epsilon = 1e-4):
    
    beta_squared = beta**2
    
    y_true = trf.cast(y_true, trf.float32)
    y_pred = trf.cast(trf.greater(trf.cast(y_pred, trf.float32), trf.constant(0.5)), trf.float32)
    
    tp = trf.reduce_sum(y_true * y_pred, axis = 1)
    fp = trf.reduce_sum(y_pred, axis = 1) - tp
    fn = trf.reduce_sum(y_true, axis = 1) - tp
    
    precision = tp/(tp+fp+epsilon)
    recall = tp/(tp+fn+epsilon)
    
    fb = (1+beta_squared)*precision*recall / (beta_squared*precision+recall+epsilon)
    return fb

In [34]:
def multi_label_acc(y_true, y_pred, epsilon = 1e-4):
  
    y_true = trf.cast(y_true, trf.float32)
    y_pred = trf.cast(trf.greater(trf.cast(y_pred, trf.float32), trf.constant(0.5)), trf.float32)
    
    tp = trf.reduce_sum(y_true * y_pred, axis = 1)
    fp = trf.reduce_sum(y_pred, axis = 1) - tp
    fn = trf.reduce_sum(y_true, axis = 1) - tp
    
    y_true = trf.cast(y_true, trf.bool)
    y_pred = trf.cast(y_pred, trf.bool)
        
    tn = trf.reduce_sum(trf.cast(trf.logical_not(y_true), trf.float32)
                       * trf.cast(trf.logical_not(y_pred), trf.float32), axis = 1)
    
    return (tp+tn)/(tp+tn+fp+fn+epsilon)

In [35]:
#defining our model
def build_model():
    model = Sequential()
    model.add(BatchNormalization(input_shape=(128, 128, 3)))
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(17, activation='sigmoid'))

    opt = Adam(lr=1e-4)
    
    model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=[multi_label_acc, fbeta])

    return model

In [36]:
#modelcheckpoint is set to monitor the model using validation fbeta score and save the best only
save_best_check_point = ModelCheckpoint(filepath = 'best_model.hdf5', 
                                        monitor = 'val_fbeta',
                                        mode = 'max',
                                        save_best_only = True,
                                        save_weights_only = True)

In [37]:
#initializing imagedatagenerator with a validation split of 0.2
train_image_gen = ImageDataGenerator(rescale = 1/255, validation_split = 0.2)

#generating train data generator which is 80% of the train dataset
#note that a generator contains both features and target of the data
train_generator = train_image_gen.flow_from_dataframe(dataframe=train,
                                                directory ="/kaggle/input/planets-dataset/planet/planet/train-jpg",  
                                                x_col="image_name", y_col=columns, subset="training", 
                                                batch_size=16,seed=2021, shuffle=True, 
                                                class_mode="raw", target_size=(128,128))

#generating validation data which is expected to be 20% of the train dataset since validation split is 0.2
val_generator = train_image_gen.flow_from_dataframe(dataframe=train,
                                                directory ="/kaggle/input/planets-dataset/planet/planet/train-jpg",  
                                                x_col="image_name", y_col=columns, subset="validation", 
                                                batch_size=16,seed=2021, shuffle=True, 
                                                class_mode="raw", target_size=(128,128))

Found 32384 validated image filenames.
Found 8095 validated image filenames.


In [38]:
import numpy as np
#setting up step size for training and validation image data
step_train_size = int(np.ceil(train_generator.samples / train_generator.batch_size))
step_val_size = int(np.ceil(val_generator.samples / val_generator.batch_size))

In [39]:
#initialize the model
model1 = build_model()

In [40]:
# Preview the model architecture
model1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_3 (Batc  (None, 128, 128, 3)      12        
 hNormalization)                                                 
                                                                 
 conv2d_24 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 conv2d_25 (Conv2D)          (None, 126, 126, 32)      9248      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 63, 63, 32)       0         
 g2D)                                                            
                                                                 
 dropout_15 (Dropout)        (None, 63, 63, 32)        0         
                                                                 
 conv2d_26 (Conv2D)          (None, 63, 63, 64)       

In [41]:
#fitting our model using the parameters already defined 
model1.fit(x = train_generator, 
           steps_per_epoch = step_train_size, 
           validation_data = val_generator, 
           validation_steps = step_val_size,epochs = 10, 
           callbacks=[save_best_check_point])

Epoch 1/10


2024-06-03 09:02:34.269623: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_3/dropout_15/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2024/2024 [==============================] - 199s 93ms/step - loss: 0.2142 - multi_label_acc: 0.9119 - fbeta: 0.6749 - val_loss: 0.1948 - val_multi_label_acc: 0.9269 - val_fbeta: 0.7116
Epoch 2/10
2024/2024 [==============================] - 115s 57ms/step - loss: 0.1755 - multi_label_acc: 0.9305 - fbeta: 0.7573 - val_loss: 0.1552 - val_multi_label_acc: 0.9373 - val_fbeta: 0.7875
Epoch 3/10
2024/2024 [==============================] - 111s 55ms/step - loss: 0.1604 - multi_label_acc: 0.9361 - fbeta: 0.7810 - val_loss: 0.1525 - val_multi_label_acc: 0.9392 - val_fbeta: 0.7887
Epoch 4/10
2024/2024 [==============================] - 112s 55ms/step - loss: 0.1543 - multi_label_acc: 0.9380 - fbeta: 0.7909 - val_loss: 0.1411 - val_multi_label_acc: 0.9432 - val_fbeta: 0.8157
Epoch 5/10
2024/2024 [==============================] - 111s 55ms/step - loss: 0.1487 - multi_label_acc: 0.9408 - fbeta: 0.8049 - val_loss: 0.1369 - val_multi_label_acc: 0.9458 - val_fbeta: 0.8184
Epoch 6/10
2024/2024 [====

In [42]:
#initializing a second model to make predictions
model2 = build_model()

In [43]:
#loading in the weights of the trained model
model2.load_weights('best_model.hdf5')

In [44]:
##adding .jpg extension to image name in the sample submission file
sample_submission = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/sample_submission.csv')
sample_submission1 = sample_submission.copy()
sample_submission1['image_name'] = sample_submission1['image_name'].apply(lambda x: '{}.jpg'.format(x))
sample_submission1.head()

,image_name,tags
0,test_0.jpg,primary clear agriculture road water
1,test_1.jpg,primary clear agriculture road water
2,test_2.jpg,primary clear agriculture road water
3,test_3.jpg,primary clear agriculture road water
4,test_4.jpg,primary clear agriculture road water


In [45]:
# Divide the sample submission file into two splits,
# first test_df which contains the first 40669 images 
test_df = sample_submission1.iloc[:40669]['image_name'].reset_index().drop('index', axis =1)
test_df.head()

,image_name
0,test_0.jpg
1,test_1.jpg
2,test_2.jpg
3,test_3.jpg
4,test_4.jpg


In [46]:
#initialize imagedatagenerator for the test images and also rescaling
test_image_gen = ImageDataGenerator(rescale = 1/255)

#creating a generator for the images found in the first test image files
test_generator = test_image_gen.flow_from_dataframe(dataframe=test_df, 
                                                directory="/kaggle/input/planets-dataset/planet/planet/test-jpg", 
                                                x_col="image_name", 
                                                y_col=None, 
                                                batch_size=16, 
                                                shuffle=False, 
                                                class_mode=None, 
                                                target_size=(128,128))

step_test_size = int(np.ceil(test_generator.samples/test_generator.batch_size))



Found 40669 validated image filenames.


In [47]:
#first, we reset the test generator to avoid shuffling of index as we want it to be orderly
test_generator.reset()
pred = model2.predict(test_generator, steps = step_test_size, verbose = 1)

2542/2542 [==============================] - 187s 73ms/step


In [48]:
#this is to get the filenames in the generator using the attribute .filenames
file_names = test_generator.filenames

#convert the predicted values to a dataframe and join two labels together if the probability of occurrance 
#of the label is greater than 0.5 
pred_tags = pd.DataFrame(pred)
pred_tags = pred_tags.apply(lambda x: ' '.join(np.array(labels)[x>0.5]), axis = 1)

#then the result should look like this 
result1 = pd.DataFrame({'image_name': file_names, 'tags': pred_tags})
result1.head()

,image_name,tags
0,test_0.jpg,clear primary
1,test_1.jpg,clear primary
2,test_2.jpg,primary partly_cloudy
3,test_3.jpg,clear primary
4,test_4.jpg,cloudy partly_cloudy


In [49]:
#additional test dataset
add_test_df = sample_submission1.iloc[40669:]['image_name'].reset_index().drop('index', axis =1)
add_test_df.head()

,image_name
0,file_0.jpg
1,file_1.jpg
2,file_10.jpg
3,file_100.jpg
4,file_1000.jpg


In [50]:
#creating a generator for the additional test image files
add_test_generator = test_image_gen.flow_from_dataframe(dataframe = add_test_df, 
                                                    directory ="/kaggle/input/planets-dataset/test-jpg-additional/test-jpg-additional", 
                                                    x_col="image_name", 
                                                    y_col=None, 
                                                    batch_size=16, 
                                                    shuffle=False, 
                                                    class_mode=None, 
                                                    target_size=(128,128))


step_test_size2 = int(np.ceil(add_test_generator.samples/add_test_generator.batch_size))

Found 20522 validated image filenames.


In [52]:
#we reset the generator to avoid shuffling, then make prediction on the generator
add_test_generator.reset()
add_pred = model2.predict(add_test_generator, steps = step_test_size2, verbose = 1)

1283/1283 [==============================] - 94s 73ms/step


In [53]:
#this is to get the filenames in the generator using the attribute .filenames
file_names = add_test_generator.filenames

#convert the predicted values to a dataframe and join two labels together if the probability of occurrance 
#of the label is greater than 0.5
add_pred_tags = pd.DataFrame(add_pred)
add_pred_tags = add_pred_tags.apply(lambda x: ''.join(np.array(labels)[x>0.5]), axis = 1)

#then the result should look like this
result2 = pd.DataFrame({'image_name': file_names, 'tags': add_pred_tags})
result2.head()

,image_name,tags
0,file_0.jpg,clearprimary
1,file_1.jpg,roadprimaryagriculturepartly_cloudy
2,file_10.jpg,roadclearprimarywater
3,file_100.jpg,clearprimaryagriculture
4,file_1000.jpg,clearprimary


In [54]:
#for the final result of the predicted tags for the test images,
# we need to concat the first and second results in 
#that order to avoid shuffling the index
last_result = pd.concat([result1, result2])

last_result = last_result.reset_index().drop('index', axis =1)

print(last_result.shape)
#print the final result
last_result.head()

(61191, 2)


,image_name,tags
0,test_0.jpg,clear primary
1,test_1.jpg,clear primary
2,test_2.jpg,primary partly_cloudy
3,test_3.jpg,clear primary
4,test_4.jpg,cloudy partly_cloudy


In [55]:
#we need to remove the .jpg extension from the image_name of the
# last_result because from the sample submission the 
#extension was not there, we added it for easy manipulation of the data.
last_result['image_name'] = last_result['image_name'].apply(lambda x: x[:-4])
last_result.head()

,image_name,tags
0,test_0,clear primary
1,test_1,clear primary
2,test_2,primary partly_cloudy
3,test_3,clear primary
4,test_4,cloudy partly_cloudy


In [56]:
last_result.to_csv('submission1.csv', index = False)